In [92]:
df = pd.read_csv('Infrared.csv')
df

,Gender,Age,Ethnicity,T_atm,Humidity,Distance,T_offset1,Max1R13_1,Max1L13_1,aveAllR13_1,...,T_FHRC1,T_FHLC1,T_FHBC1,T_FHTC1,T_FH_Max1,T_FHC_Max1,T_Max1,T_OR1,T_OR_Max1,aveOralM
0,Male,41-50,White,24.0,28.0,0.8,0.7025,35.0300,35.3775,34.4000,...,33.4775,33.3725,33.4925,33.0025,34.5300,34.0075,35.6925,35.6350,35.6525,36.59
1,Female,31-40,Black or African-American,24.0,26.0,0.8,0.7800,34.5500,34.5200,33.9300,...,34.0550,33.6775,33.9700,34.0025,34.6825,34.6600,35.1750,35.0925,35.1075,37.19
2,Female,21-30,White,24.0,26.0,0.8,0.8625,35.6525,35.5175,34.2775,...,34.8275,34.6475,34.8200,34.6700,35.3450,35.2225,35.9125,35.8600,35.8850,37.34
3,Female,21-30,Black or African-American,24.0,27.0,0.8,0.9300,35.2225,35.6125,34.3850,...,34.4225,34.6550,34.3025,34.9175,35.6025,35.3150,35.7200,34.9650,34.9825,37.09
4,Male,18-20,White,24.0,27.0,0.8,0.8950,35.5450,35.6650,34.9100,...,35.1600,34.3975,34.6700,33.8275,35.4175,35.3725,35.8950,35.5875,35.6175,37.04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1015,Female,21-25,Asian,25.7,50.8,0.6,1.2225,35.6425,35.6525,34.8575,...,35.3475,35.4000,35.1375,35.2750,35.8525,35.7475,36.0675,35.6775,35.7100,36.99
1016,Female,21-25,White,25.7,50.8,0.6,1.4675,35.9825,35.7575,35.4275,...,35.2175,35.2200,35.2075,35.0700,35.7650,35.5525,36.5000,36.4525,36.4900,37.19
1017,Female,18-20,Black or African-American,28.0,24.3,0.6,0.1300,36.4075,36.3400,35.8700,...,35.2400,35.2275,35.3675,35.3425,36.3750,35.7100,36.5350,35.9650,35.9975,37.59
1018,Male,26-30,Hispanic/Latino,25.0,39.8,0.6,1.2450,35.8150,35.5250,34.2950,...,35.0200,34.9250,34.7150,34.5950,35.4150,35.3100,35.8600,35.4150,35.4350,37.29


In [93]:
import torch
import torch.nn as nn
import torch.optim as optim
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
import pandas as pd

In [94]:
data = pd.read_csv("Infrared.csv")

In [95]:
imputer = SimpleImputer(strategy='mean')
data.iloc[:, 3:] = imputer.fit_transform(data.iloc[:, 3:])

In [96]:
label_encoders = {}
for col in ["Gender", "Age", "Ethnicity"]:
    le = LabelEncoder()
    data[col] = le.fit_transform(data[col])
    label_encoders[col] = le

In [97]:
features = [col for col in data.columns if col != "aveOralM"]
target = "aveOralM"

In [98]:
scaler = StandardScaler()
data[features] = scaler.fit_transform(data[features])

In [99]:
X_train, X_test, y_train, y_test = train_test_split(data[features], data[target], test_size=0.2, random_state=42)
X_train, X_test, y_train, y_test = map(torch.tensor, (X_train.values, X_test.values, y_train.values, y_test.values))
X_train, X_test = X_train.float(), X_test.float()
y_train, y_test = y_train.float(), y_test.float()

In [186]:
class RegressionModel(nn.Module):
    def __init__(self):
        super(RegressionModel, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(len(features), 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
        )
    
    def forward(self, x):
        return self.fc(x)

model = RegressionModel()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.004)

In [188]:
for epoch in range(500):
    model.train()
    optimizer.zero_grad()
    predictions = model(X_train)
    loss = criterion(predictions, y_train)
    loss.backward()
    optimizer.step()
    
    if epoch % 100 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item()}")

Epoch 0, Loss: 0.2907562553882599
Epoch 100, Loss: 0.2825714647769928
Epoch 200, Loss: 0.3603743612766266
Epoch 300, Loss: 0.2912067174911499
Epoch 400, Loss: 0.27458861470222473


In [189]:
model.eval()
y_pred = model(X_test).detach().numpy().flatten()
mse = mean_squared_error(y_test.numpy(), y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test.numpy(), y_pred)
print(f"PyTorch Model - MSE: {mse:.6f}, RMSE: {rmse:.6f}, R²: {r2:.6f}")

PyTorch Model - MSE: 0.619055, RMSE: 0.786800, R²: -1.939870


In [190]:
tf_model = keras.Sequential([
    layers.Input(shape=(len(features),)),
    layers.Dense(512, activation='relu'),
    layers.Dense(256, activation='relu'),
    layers.Dense(128, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(1)
])

In [191]:
tf_model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0004), loss='mse', metrics=['mse'])

In [192]:
tf_model.fit(X_train.numpy(), y_train.numpy(), epochs=500, batch_size=32, verbose=0)

In [193]:
y_pred_tf = tf_model.predict(X_test.numpy()).flatten()
mse_tf = mean_squared_error(y_test, y_pred_tf)
rmse_tf = np.sqrt(mse_tf)
r2_tf = r2_score(y_test, y_pred_tf)
print(f"TensorFlow Model - MSE: {mse_tf:.6f}, RMSE: {rmse_tf:.6f}, R²: {r2_tf:.6f}")

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step 
TensorFlow Model - MSE: 0.313830, RMSE: 0.560205, R²: -0.490367


### Analisis Perbandingan Model PyTorch dan TensorFlow 

1. Latar Belakang
Membangun model regresi menggunakan deep learning guna memprediksi nilai aveOralM berdasarkan fitur yang tersedia dalam dataset Infrared.csv. Dua model dikembangkan menggunakan framework PyTorch dan TensorFlow untuk membandingkan performanya dalam menyelesaikan tugas prediksi ini.  

Dataset mengalami beberapa tahap preprocessing, seperti penanganan nilai hilang, encoding variabel kategorikal, normalisasi fitur numerik, dan pembagian data menjadi training set (80%) dan testing set (20%).  

---

2. Arsitektur Model 
Kedua model memiliki arsitektur yang mirip, terdiri dari lima hidden layers dengan aktivasi ReLU dan satu neuron di layer output untuk menghasilkan prediksi.  

Metode optimasi yang digunakan adalah Adam optimizer dengan learning rate sebesar 0.004, dan kedua model dilatih selama 500 epoch.  

---

3. Evaluasi dan Hasil Model  
Model dievaluasi menggunakan tiga metrik utama:  
- Mean Squared Error (MSE) → Semakin kecil nilainya, semakin baik model dalam memprediksi target.  
- Root Mean Squared Error (RMSE) → Mengukur kesalahan model dalam skala yang sama dengan target asli.  
- R-squared (R²) → Mengukur seberapa baik model menjelaskan variasi data, dengan nilai ideal mendekati **1.0**.  

Hasil Evaluasi: 

| Model         | MSE      | RMSE     | R²         |
|--------------|---------|---------|-----------|
| PyTorch  | 0.6191  | 0.7868  | -1.9399   |
| TensorFlow | 0.3138  | 0.5602  | -0.4904   |

---
4. Penjelasan Persamaan Matematika dalam Evaluasi Model 

---

**1. Mean Squared Error (MSE)**  
**Persamaan:**  
\[
MSE = \frac{1}{n} \sum_{i=1}^{n} (y_i - \hat{y}_i)^2
\]  
**Penjelasan:**  
- \( y_i \) adalah nilai aktual dari target.  
- \( \hat{y}_i \) adalah nilai prediksi model.  
- \( n \) adalah jumlah sampel dalam dataset.  
- Simbol \( \sum \) berarti menjumlahkan seluruh perbedaan kuadrat antara nilai aktual dan prediksi.  
- Nilai MSE yang lebih kecil menunjukkan kesalahan prediksi yang lebih rendah.  

Dalam hasil proyek ini:  
- PyTorch Model → MSE: 0.6191  
- TensorFlow Model → MSE: 0.3138 
Karena model TensorFlow memiliki MSE lebih rendah, maka ia lebih akurat dibandingkan PyTorch.  

---

**2. Root Mean Squared Error (RMSE)**  
**Persamaan:**  
\[
RMSE = \sqrt{MSE} = \sqrt{\frac{1}{n} \sum_{i=1}^{n} (y_i - \hat{y}_i)^2}
\]  
**Penjelasan:**  
- RMSE adalah akar kuadrat dari MSE.  
- Fungsi utama RMSE adalah untuk memberikan kesalahan dalam satuan yang sama dengan data asli, sehingga lebih mudah untuk diinterpretasikan.  
- Semakin kecil RMSE, semakin baik model dalam memprediksi data.  

Dalam hasil proyek ini:  
- PyTorch Model → RMSE: 0.7868 
- TensorFlow Model → RMSE: 0.5602  
Model TensorFlow memiliki RMSE lebih kecil, sehingga lebih unggul dibandingkan PyTorch.  

---

**3. R-squared (R²) atau Koefisien Determinasi**  
**Persamaan:**  
\[
R^2 = 1 - \frac{\sum_{i=1}^{n} (y_i - \hat{y}_i)^2}{\sum_{i=1}^{n} (y_i - \bar{y})^2}
\]  
**Penjelasan:**  
- \( \bar{y} \) adalah rata-rata dari semua nilai target \( y \).  
- Pembilang (\( \sum (y_i - \hat{y}_i)^2 \)) adalah jumlah kuadrat dari kesalahan prediksi (SSE - Sum of Squared Errors).  
- Penyebut (\( \sum (y_i - \bar{y})^2 \)) adalah jumlah kuadrat total dari data asli (SST - Sum of Squares Total).  
- R² mengukur seberapa baik model menjelaskan variasi dalam data.  
  - Nilai R² = 1 berarti model sempurna.  
  - Nilai R² = 0 berarti model tidak lebih baik dari tebakan rata-rata.  
  - Nilai R² negatif berarti model lebih buruk daripada hanya menggunakan rata-rata target.  

Dalam hasil proyek ini:  
- PyTorch Model → R²: -1.9399  
- TensorFlow Model → R²: -0.4904  
Karena R² model TensorFlow lebih mendekati nol dibandingkan PyTorch, maka TensorFlow lebih baik dalam menjelaskan variasi data, meskipun belum optimal.  

---

5. Analisis Perbandingan PyTorch vs. TensorFlow  

Akurasi Model  
Dari hasil evaluasi, model TensorFlow memiliki MSE yang lebih rendah dibandingkan model PyTorch. Ini menunjukkan bahwa model TensorFlow lebih baik dalam memprediksi nilai target dengan kesalahan yang lebih kecil.  

Selain itu, nilai R² dari model TensorFlow lebih baik dibandingkan PyTorch**. R² yang negatif menandakan bahwa model masih belum optimal dalam menjelaskan variasi data, tetapi TensorFlow lebih dekat ke 0, yang berarti modelnya lebih baik dibandingkan PyTorch dalam menangkap pola dari dataset.  

6. Kesimpulan

Berdasarkan hasil evaluasi, model TensorFlow menunjukkan performa yang lebih baik dibandingkan PyTorch dalam tugas regresi ini. Hal ini terlihat dari nilai MSE yang lebih rendah (0.3138 vs. 0.6191) dan RMSE yang lebih kecil (0.5602 vs. 0.7868), yang menandakan bahwa model TensorFlow mampu membuat prediksi dengan kesalahan yang lebih kecil dibandingkan model PyTorch. Selain itu, nilai R² pada model TensorFlow (-0.4904) lebih baik dibandingkan PyTorch (-1.9399), yang mengindikasikan bahwa model TensorFlow lebih mampu menjelaskan variasi data dibandingkan PyTorch, meskipun masih belum optimal. Secara keseluruhan, model TensorFlow lebih unggul dalam mempelajari pola dari dataset dan menghasilkan prediksi yang lebih akurat dibandingkan model PyTorch. Perlu dilakukan optimasi lebih lanjut agar model bisa menghasilkan hasil yang lebih kompetitif dibandingkan TensorFlow.  